# i.MX 95 AHAB Load M7 Application to TCM

This notebook describes how to build a bootable AHAB image with M7 application, and how to load it to TCM (Tightly Coupled Memory).

## 1. Prerequisites

- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
- This demo was tested with i.MX 95 EVK board with LPDDR4 memory and A0 chip revision.

## 1.1 Images preparation

- Obtain all the necessary binaries and put them into inputs directory

## 1.2 Requirements

Download these files from the `https://www.nxp.com/lgfiles/NMG/MAD/YOCTO/<package name>` and put them into the inputs directory.

Primary image container set:
- ELE firmware
- lpddr5 or lpddr4 firmware files with OEI firmware
- CM33 OEI TCM
- CM33 System manager image
- M7 application image


In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. AHAB Image


### 2.1 AHAB Template

We can generate the template using the nxpimage ahab get-template command. The command will generate a template. The template is a YAML file that contains the AHAB header and the AHAB container. The AHAB header contains the information about the image, such as the version, the number of containers, and the signature. The AHAB container contains the information about the image, such as the image type, the image version, the image size, and the image signature.

The following command generates the template:

```bash
nxpimage ahab get-template -f mimx9596 -o ahab_template.yaml
```

### 2.2 Exporting of the AHAB image

The AHAB image can be exported using the nxpimage ahab export command. The command will create the AHAB image from the template. The following command creates the AHAB image:

```bash
nxpimage ahab export -c ahab_template.yaml 
```


Image container set consists of the following images:

- ELE firmware
- DDR firmware with OEI
- System manager
- M7 application image

In [4]:
YamlDiffWidget("inputs/mx95_tcm.diffc").html

nxpimage ahab get-template -r a0 -f mimx9596 -o workspace/ahab_template.yaml --force
Creating workspace/ahab_template.yaml template file.


In [4]:
# EXPORT U-BOOT IMAGE
%! nxpimage -v ahab export --config inputs/mx95_tcm.yaml

nxpimage -v ahab export --config inputs/mx95_tcm.yaml
INFO:spsdk.apps.nxpimage:Created AHAB Image:
Name:      AHAB Image
Starts:    0x0
Ends:      0xb2bff
Size:      Size: 715.0 kiB; 732,160 B
Alignment: 512 B
Execution Start Address: Not defined
Pattern:zeros
AHAB Image for mimx9596_a0

INFO:spsdk.apps.nxpimage:Created AHAB Image memory map:

+==0x0000_0000= AHAB Image ==========================+
|             Size: 715.0 kiB; 732,160 B             |
|             AHAB Image for mimx9596_a0             |
|                   Pattern: zeros                   |
|+==0x0000_0000= AHAB Containers ===================+|
||              Size: 8.0 kiB; 8,192 B              ||
||              AHAB Containers block               ||
||                  Pattern: zeros                  ||
||+==0x0000_0000= AHAB Container 0 ================+||
|||                  Size: 544 B                   |||
|||         AHAB Container for nxp_SWver:0         |||
||+==0x0000_021f=================================

In [ ]:
# Set the boot mode to Cortex-M Serial Downloader 1001 and download the files using the UUU tool

%! nxpuuu $VERBOSITY run "SDPS[-t 10000]: boot -f outputs/flash.bin"

nxpuuu -v write -f mimx9596 -b emmc outputs/flash.bin 
SDPS: boot -f outputs/flash.bin


SDPV: write -f outputs/flash.bin -skipspl


SDPV: jump


Done
